In [26]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

importlib.reload(utils)
importlib.reload(rt)

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

In [27]:
air_joined = pd.read_parquet('airtable_joined.parquet')

In [28]:
analysis_date = dt.date(2022, 2, 8) ##tuesday, new tables

In [29]:
colorscale = branca.colormap.step.RdYlGn_10.scale(vmin=0, vmax=30)
colorscale.caption = "Speed (miles per hour)"

In [30]:
# d3 = [273, 372, 221]

In [31]:
ran_operators = [int(filename.split('_')[0]) for filename in os.listdir('speedmaps') if filename[0] != '.']

In [32]:
pbar = tqdm()

0it [00:00, ?it/s]

In [36]:
# for agency in air_joined.calitp_itp_id.unique():
for agency in [294]:
    not_ran_operators = []
    # if agency in ran_operators:
    #     print(f'already ran: {agency}')
    #     continue
    print(f'calculating for agency: {agency}...')
    try:
        rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)
        rt_day.set_filter(start_time='15:00', end_time='19:00')
        rt_day.filter_formatted = ', PM Peak, Feb 08 (Tue)' ##TODO fix hardcode
        rt_day.agency_name = (air_joined >> filter(_.calitp_itp_id == agency)).Name.iloc[0]
        m = rt_day.segment_speed_map(how='low_speeds', colorscale=colorscale, size = [1300, 700])
        m.save(f'./speedmaps/{agency}_02_08_pm_peak.html')
        print(f'complete for agency: {agency}')
    except Exception as e:
        print(f'rt failed for agency {agency}')
        not_ran_operators += [agency]
        print(e)

calculating for agency: 294...


/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:36: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:36: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a 

vehicle positions gdf must not be empty
could not generate delays for trip 3149882
time data '24:34:45' does not match format '%H:%M:%S'
could not generate delays for trip 3153140
time data '24:02:01' does not match format '%H:%M:%S'
could not generate delays for trip 3149013
time data '26:03:00' does not match format '%H:%M:%S'
could not generate delays for trip 3124081
time data '24:07:00' does not match format '%H:%M:%S'
could not generate delays for trip 3124080
time data '24:41:00' does not match format '%H:%M:%S'
could not generate delays for trip 3124050
time data '25:29:23' does not match format '%H:%M:%S'
could not generate delays for trip 3149250
time data '25:04:52' does not match format '%H:%M:%S'
could not generate delays for trip 3148112
time data '24:20:09' does not match format '%H:%M:%S'
could not generate delays for trip 3148015
Columns must be same length as key
could not generate delays for trip 3132229
time data '24:09:16' does not match format '%H:%M:%S'
could not

TopologyException: assigned depths do not match at -162134.71517185928 -83737.066755124557
/opt/conda/lib/python3.9/site-packages/shapely/ops.py:634: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis.py:509: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



complete for agency: 294


In [8]:
# air_joined['url'] = air_joined.apply(lambda x:
#                     f'http://docs.calitp.org/data-analyses/rt_delay/speedmaps/{x.calitp_itp_id}_02_08_pm_peak.html',
#                                     axis = 1)

# air_joined.to_csv('linked.csv')

In [37]:
dt.datetime.fromtimestamp(1644622458)

datetime.datetime(2022, 2, 11, 15, 34, 18)